In [1]:
require 'rest-client'
require 'json' # to handle JSON format

##------------------- Mark's fuction to prevent web accessing errors --------------------
#puts "now I load the fetch function"
def fetch(url, headers = {accept: "*/*"}, user = "", pass="")
  response = RestClient::Request.execute({
    method: :get,
    url: url.to_s,
    user: user,
    password: pass,
    headers: headers})
  return response
  
  rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.response
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
  rescue RestClient::Exception => e
    $stderr.puts e.response
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
  rescue Exception => e
    $stderr.puts e
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
end


#####
#####      WEB ACCESSING FUNCTIONS 
#####

#------------------- get protein ID --------------------
def get_protID(my_gene)
  address="http://togows.org/entry/ebi-uniprot/#{my_gene}/accessions.json"
  response = fetch(address)  
  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = JSON.parse(response.body)
    return data[0]
  end
end

#------------------- get KEGG annotations --------------------
def get_KEGG(my_gene)
  address = "http://togows.org/entry/kegg-genes/ath:#{my_gene}/pathways.json"
  my_KEGG_list=Array.new
  response = fetch(address)  
  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = JSON.parse(response.body)
    my_KEGG_list.push(data[0].to_a)
    if data[0].length != 0
      my_KEGG_list = data[0].to_a
    end
    return my_KEGG_list
  else 
    return Array.new
  end
end


#------------------- get GO annotations --------------------
def get_GO(my_gene)
  address = "http://togows.dbcls.jp/entry/uniprot/#{my_gene}/dr.json"
  my_GO_list=Array.new
  response = fetch(address)  
  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = JSON.parse(response.body)
    if data[0]["GO"]
      data[0]["GO"].each do |go| 
      if go[1] =~ /P:/#if its a biological process it will have the key 'P'
        my_GO_list.push(go[0..1])
      end
    end
      return my_GO_list
    end
  else 
    return Array.new
  end
end


#------------------- get gene interactions --------------------

def get_interactions(my_gene,all_genes)
  address="http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/query/#{my_gene}?format=tab25"
  response = fetch(address)
  interactions_array=Array.new
  #puts "get_interactions function has been called"
  
  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = body.split("\n").to_a #first I split the rows into different arrays
    (0..data.length-1).each do |i|

      data[i] = data[i].split("\t") #then I create an array's element per each tab separated value 
      data[i] = data[i][4..5] #these columns of the tab25 format cointain the gene locus name

      data.each do |item|
        (0..item.length-1).each do |k|
          interactions_array.push(item[k].scan(/A[Tt]\d[Gg]\d\d\d\d\d/)) #find the agi code in the text, retrieve it and add it to the array
        end
      end
    end
    
    interactions_array = interactions_array.flatten.uniq  #unnest the arrray, delete repetitions and interactions with itself
    interactions_array.map!(&:upcase) #make all the codes uppercase so that I can compare them latter
    interactions_array = interactions_array- [my_gene.upcase] #dont include my own gene as an interaction
    #puts "interactions_array created"
    
    return interactions_array
    
  end
end


:get_interactions

In [71]:
#----------------------- CREATE GENE CLASS ----------------------------
class Gene
  attr_accessor :gene_id # AGI gene ID code
  attr_accessor :prot_id # UniProt IDs
  attr_accessor :kegg # Array with KEGG's ID and pathway name
  attr_accessor :go # Array with GO's ID and term name
  attr_accessor :interactions # Array with interactions
  @@all_objects = Array.new
  @@all_genes = Array.new
  
  
  def initialize (params = {}) 
##    agi_code = params.fetch(:agi, "AT0G00000")
##    if (/A[Tt]\d[Gg]\d\d\d\d\d/).match(agi_code) #regular expresion for gene identifier format
##      @gene_id = agi_code
##    else 
##      raise "\n Error: invalid gene ID format \n Make sure that ALL gene IDs have AT0G00000 format"
##    end
    @gene_id = params.fetch(:gene_id, "AT0G00000")
    unless @gene_id.match(/A[Tt]\d[Gg]\d\d\d\d\d/) #if the code does not match the regular expression, raise an error
      raise "\n Error: invalid gene ID format: #{@gene_id} \n Make sure ALL gene IDs have ATxGxxxxx format"
    end
    @prot_id = params.fetch(:prot_id, Array.new) #UniProt IDs
    @kegg = params.fetch(:kegg, Array.new) #Array with KEGG's ID and pathway name
    @go = params.fetch(:go, Array.new) #Array with GO's ID and term name
    @interactions = params.fetch(:interactions, Array.new) #Array with GO's ID and term name
    
    @@all_objects << self
    @@all_genes.push(@gene_id.upcase)
    
  end
  
  def show_all_objects
    return @@all_objects
  end
  
  def all_genes
    return @@all_genes
  end
  
  ## METHODS TO SHOW ANNOTATED INFO GIVEN A GENE ID

  
  def show_kegg(id) 
    @@all_objects.each do |object|
      #puts object.gene_id
      if object.gene_id == id
        res=object.kegg
        return res
      end
    end
  end
  
  def show_go(id)
    @@all_objects.each do |object|
      if object.gene_id == id
        res=object.go
        return res
      end
    end
  end
  
  def show_interactions(id)
    @@all_objects.each do |object|
      if object.gene_id == id
        res=object.interactions
        return res
      end
    end
  end
  
  

  def get_info
    puts "Gene's ID : #{self.gene_id}"
    puts "Proteins's UniProt name: #{self.prot_id}"
    puts "KEGG's ID and pathway name: #{self.kegg}"
    puts "GO's ID and term name: #{self.go}"
    puts "Protein interactions: #{self.interactions}"
  end
  
  
  def get_network
  ## first level interactions
    int_L1=Array.new    
    @@all_objects.each do |gene|
       if gene.interactions != [] # if there are interacions
         int_L1.push([gene.gene_id,gene.interactions].flatten) # add that interaction as [my_gene,interaction1,interaction2, ...] 
       end
     end
    
    #puts "L1 is #{int_L1}"
    
    ## second level interactions
    int_L2=Array.new
    int_L1.each do |gene|  # takes something like [my_gene, interaction1, interaction2, ...]
    
      gene[1..-1].each do |interaction| #takes each item of [interaction1, interaction2, ...]
        int_list=[]
        
        int_L1.each do |item| # per set of [my_gene, [interaction1, interaction2, ...]]
          if item[1..-1].include? interaction # if [interaction1, interaction2, ...] includes my_gene
            int_list.push([item[0]]) #add my_gene to the list
          end
        end
        
        int_list.flatten!.uniq! # un-nest arrays and remove duplicates
        if int_list.length > 1 #there is interaction with 2 genes and therefore is part of a network
          int_L2.push([interaction,int_list].flatten)
        end
        
      end
    end
    
    #puts ""
    #puts "L2 is #{int_L2}"
    
    ## merge and make that array a local variable so it can be called later on
    all=[int_L1,int_L2].flatten!(1)
    #puts "all is #{all}"
    #puts ""
    
     
    ## create network's arrays
    
    networks=Array.new
    all.each do |item| # get one array of interactions, for example [gene1 , gene2, gene3]
      puts "item is #{item}"
      net=Array.new
      
      item.each do |elem| # get one of those genes, for example gene1
        
        (0..all.length-1).each do |i| #iterate over the arrays again (like all[0] = [gene1 , gene2, gene3])
          if all[i].any? elem #if the array contains that gene (for example gene1 is in all[0])
            net.push(all[i].flatten) # include that array of interactions to my new array 
          end
        end
      end
      net.flatten!.uniq!.sort!
      if net.length > 2 
        puts "one network is #{net}"
        puts ""
        networks.push(net)
      end
    end
    
    networks.uniq! #remove duplicated networks
    return networks
      
  end
      
  
end



:get_network

In [72]:
#------------------- LOAD INFO FROM GENE FILE --------------------

#Now I open the gene file and create some arrays that will help create the Class objects
gene_file = File.new("/Users/sara/BioCompu/Programming_challenges/Task2/ArabidopsisSubNetwork_GeneList.txt","r")

genes_list=Array.new 
gene_file.each do |agi| # iterate over the elements of gene_file
  gene_agi_code = agi.strip # delete posible "/t" or "/n" 
  gene_agi_code.upcase! #make all codes upper case to avoid errors later on
  genes_list.push(gene_agi_code) #add AGI codes to a list
end
puts ""

In [73]:
genes_list= ["AT4G27030", "AT5G54270", "AT1G21400", "AT5G19120", "AT2G13360"]

["AT4G27030", "AT5G54270", "AT1G21400", "AT5G19120", "AT2G13360"]

In [74]:
object=Array.new
genes_list.each do |item|
  interactions=Array.new
  interactions = get_interactions(item,genes_list)
  if interactions != []
    object.push('new')
    object[-1]=Gene.new(
      :gene_id => item,
      :prot_id => get_protID(item),
      :kegg => get_KEGG(item),
      :go => get_GO(item),
      :interactions => interactions
       )
    puts object[-1].get_info
  end
end

Gene's ID : AT5G19120
Proteins's UniProt name: ["Q93VG3", "A0A178UFR4", "Q0WU92", "D7LYL5"]
KEGG's ID and pathway name: [[]]
GO's ID and term name: [["GO:0030163", "P:protein catabolic process"], ["GO:0006508", "P:proteolysis"]]
Protein interactions: ["AT3G46090"]

Gene's ID : AT2G13360
Proteins's UniProt name: ["Q56YA5", "O81248", "A0A178VX39", "D7L023"]
KEGG's ID and pathway name: [["ath00250", "Alanine, aspartate and glutamate metabolism"], ["ath00260", "Glycine, serine and threonine metabolism"], ["ath00630", "Glyoxylate and dicarboxylate metabolism"], ["ath01100", "Metabolic pathways"], ["ath01110", "Biosynthesis of secondary metabolites"], ["ath01200", "Carbon metabolism"], ["ath04146", "Peroxisome"]]
GO's ID and term name: [["GO:0019265", "P:glycine biosynthetic process, by transamination of glyoxylate"], ["GO:0009853", "P:photorespiration"]]
Protein interactions: ["AT3G14415", "AT3G46090", "AT2G01950"]



["AT4G27030", "AT5G54270", "AT1G21400", "AT5G19120", "AT2G13360"]

In [85]:
if !object[11111].nil?
  puts "w"
else
  puts "q"
end

q


In [65]:
a 

[["ath00250", "Alanine, aspartate and glutamate metabolism"], ["ath00260", "Glycine, serine and threonine metabolism"], ["ath00630", "Glyoxylate and dicarboxylate metabolism"], ["ath01100", "Metabolic pathways"], ["ath01110", "Biosynthesis of secondary metabolites"], ["ath01200", "Carbon metabolism"], ["ath04146", "Peroxisome"]]

In [24]:
object[0].interactions

["AT3G46090"]

In [88]:
object[0].get_network

item is ["AT5G19120", "AT3G46090"]
one network is ["AT2G01950", "AT2G13360", "AT3G14415", "AT3G46090", "AT5G19120"]

item is ["AT2G13360", "AT3G14415", "AT3G46090", "AT2G01950"]
one network is ["AT1G10090", "AT2G01950", "AT2G13360", "AT2G47930", "AT3G14415", "AT3G15710", "AT3G46090", "AT5G01530", "AT5G19120"]

item is ["AT4G05180", "AT4G25200"]
item is ["AT4G12800", "AT2G46820"]
item is ["AT5G04140", "AT4G37930", "AT1G78300"]
one network is ["AT1G09530", "AT1G09570", "AT1G78300", "AT2G18790", "AT2G20180", "AT2G25930", "AT2G40080", "AT2G43010", "AT3G46640", "AT4G33010", "AT4G37930", "AT5G04140"]

item is ["AT1G03130", "AT2G23290", "AT4G08150", "AT4G29930", "AT4G32190", "AT5G32440", "AT3G08530", "AT4G19700", "AT3G47620", "AT5G37670", "AT5G61010", "AT3G02150", "AT3G48150", "AT1G52220"]
one network is ["AT1G01260", "AT1G03040", "AT1G03130", "AT1G03150", "AT1G04990", "AT1G05710", "AT1G08465", "AT1G08970", "AT1G09660", "AT1G09670", "AT1G10586", "AT1G14920", "AT1G15670", "AT1G20700", "AT1G209


item is ["AT4G33010", "AT1G78300"]
one network is ["AT1G09530", "AT1G09570", "AT1G78300", "AT2G18790", "AT2G20180", "AT2G25930", "AT2G40080", "AT2G43010", "AT3G46640", "AT4G33010", "AT4G37930", "AT5G04140"]

item is ["AT1G12780", "AT4G23920"]
item is ["AT2G17880", "AT3G48150", "AT3G49580", "AT4G17680"]
one network is ["AT1G01260", "AT1G03130", "AT1G09660", "AT1G09670", "AT1G14920", "AT1G15670", "AT1G22920", "AT1G52220", "AT2G01570", "AT2G17880", "AT2G23290", "AT2G23420", "AT2G33880", "AT3G02150", "AT3G06940", "AT3G08530", "AT3G16250", "AT3G24140", "AT3G47620", "AT3G48150", "AT3G49580", "AT3G53900", "AT3G61150", "AT4G02770", "AT4G08150", "AT4G17680", "AT4G19700", "AT4G22890", "AT4G29930", "AT4G30080", "AT4G32190", "AT5G24660", "AT5G32440", "AT5G37670", "AT5G61010"]

item is ["AT3G49160", "AT3G15420", "AT2G47710", "AT2G42910", "AT2G01760", "AT4G16143", "AT1G71050", "AT2G36930"]
one network is ["AT1G71050", "AT2G01760", "AT2G36930", "AT2G42910", "AT2G47710", "AT3G15420", "AT3G49160", "AT


item is ["AT1G42970", "AT2G42580", "AT1G15780"]
one network is ["AT1G15780", "AT1G42970", "AT2G42580"]

item is ["AT5G17300", "AT1G08465", "AT1G20910", "AT3G15030", "AT5G08330", "AT1G76510", "AT1G60250", "AT1G60380", "AT1G69180", "AT1G71130", "AT2G14880", "AT2G25000", "AT2G30432", "AT2G39030", "AT2G41940", "AT2G44020", "AT2G45680", "AT2G48100", "AT3G02150", "AT3G04730", "AT3G03550", "AT3G12720", "AT3G17010", "AT5G24520", "AT4G14225", "AT4G15248", "AT3G59470", "AT3G53310", "AT3G53340", "AT2G31070", "AT1G03040", "AT3G18550", "AT3G24820", "AT3G26620", "AT3G47620", "AT3G48590", "AT4G03250", "AT4G35550", "AT5G59800", "AT5G56860", "AT5G47370", "AT5G42700", "AT5G23280", "AT5G23090", "AT5G15210"]
one network is ["AT1G01260", "AT1G03040", "AT1G03130", "AT1G03150", "AT1G04990", "AT1G05710", "AT1G08465", "AT1G08970", "AT1G09660", "AT1G09670", "AT1G10586", "AT1G14920", "AT1G15670", "AT1G20700", "AT1G20910", "AT1G22920", "AT1G26780", "AT1G29910", "AT1G29920", "AT1G30490", "AT1G36060", "AT1G42050",


item is ["AT3G54050", "AT4G09570"]
one network is ["AT1G26830", "AT1G35490", "AT1G35670", "AT3G01770", "AT3G48360", "AT3G54050", "AT4G09570", "AT4G14770"]

item is ["AT3G45780", "AT5G64330", "AT4G26090"]
one network is ["AT3G45780", "AT4G26090", "AT5G64330"]

item is ["AT2G30950", "AT1G22920", "AT2G25000"]
one network is ["AT1G03040", "AT1G08465", "AT1G14920", "AT1G20910", "AT1G22920", "AT1G32060", "AT1G60250", "AT1G60380", "AT1G69180", "AT1G71130", "AT1G76510", "AT2G01570", "AT2G14880", "AT2G23420", "AT2G25000", "AT2G30432", "AT2G30950", "AT2G31070", "AT2G33880", "AT2G39030", "AT2G41940", "AT2G44020", "AT2G45680", "AT2G48100", "AT3G02150", "AT3G03550", "AT3G04730", "AT3G12720", "AT3G15030", "AT3G16250", "AT3G17010", "AT3G18550", "AT3G24820", "AT3G26620", "AT3G26650", "AT3G47620", "AT3G48150", "AT3G48590", "AT3G53310", "AT3G53340", "AT3G53900", "AT3G59470", "AT3G62410", "AT4G03250", "AT4G14225", "AT4G15248", "AT4G35550", "AT5G08330", "AT5G15210", "AT5G17300", "AT5G23090", "AT5G23280",


item is ["AT5G43750", "AT1G17280", "AT1G14360", "AT5G35460", "AT5G47180", "AT1G29060", "AT4G20790", "AT2G22425", "AT3G10640", "AT1G70760", "AT2G43420", "AT3G12180", "AT5G04990"]
one network is ["AT1G01230", "AT1G01620", "AT1G04310", "AT1G07860", "AT1G08590", "AT1G10030", "AT1G10470", "AT1G11915", "AT1G14360", "AT1G16170", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G30080", "AT1G31420", "AT1G32210", "AT1G33100", "AT1G36050", "AT1G44960", "AT1G47640", "AT1G49230", "AT1G54320", "AT1G54350", "AT1G57540", "AT1G57550", "AT1G60940", "AT1G60950", "AT1G62200", "AT1G66340", "AT1G68380", "AT1G69870", "AT1G70760", "AT1G71140", "AT1G72300", "AT1G74370", "AT1G77110", "AT1G77350", "AT1G77380", "AT2G01420", "AT2G01970", "AT2G02020", "AT2G03620", "AT2G05620", "AT2G15970", "AT2G19580", "AT2G22425", "AT2G23093", "AT2G23095", "AT2G24170", "AT2G26180", "AT2G28250", "AT2G28960", "AT2G29020", "AT2G29180", "AT2G32380", "AT2G33250", "AT2G34250", "AT2G35520", "AT2G37360", "AT2G38120",

...


item is ["AT2G22425", "AT4G22890", "AT5G43750"]
one network is ["AT1G01260", "AT1G09660", "AT1G09670", "AT1G14360", "AT1G15670", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G60950", "AT1G69870", "AT1G70760", "AT2G05620", "AT2G22425", "AT2G23290", "AT2G43420", "AT3G06940", "AT3G08530", "AT3G10640", "AT3G12180", "AT3G24140", "AT3G28220", "AT3G47620", "AT3G48150", "AT3G49580", "AT3G61150", "AT3G61260", "AT4G02770", "AT4G17680", "AT4G19700", "AT4G20790", "AT4G22890", "AT4G29930", "AT4G30080", "AT4G37680", "AT4G38690", "AT5G04990", "AT5G24660", "AT5G25752", "AT5G32440", "AT5G35460", "AT5G37670", "AT5G38990", "AT5G43750", "AT5G47180", "AT5G59650", "AT5G61010", "AT5G66190"]

item is ["AT3G12180", "AT4G22890", "AT1G69870", "AT5G43750", "AT3G61260", "AT5G38990"]
one network is ["AT1G01260", "AT1G04310", "AT1G07860", "AT1G09660", "AT1G09670", "AT1G10470", "AT1G14360", "AT1G15670", "AT1G16170", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G47640",


item is ["AT5G06320", "AT1G69870", "AT5G38990"]
one network is ["AT1G04310", "AT1G07860", "AT1G16170", "AT1G17280", "AT1G29060", "AT1G47640", "AT1G60940", "AT1G69870", "AT1G71140", "AT1G77350", "AT2G26180", "AT2G28250", "AT2G32380", "AT2G41705", "AT3G10980", "AT3G12180", "AT3G17000", "AT3G21580", "AT3G26370", "AT3G28220", "AT3G45290", "AT3G52070", "AT3G61260", "AT3G66654", "AT4G14455", "AT4G20790", "AT4G22890", "AT4G30500", "AT4G30850", "AT4G37680", "AT5G06320", "AT5G11890", "AT5G38990", "AT5G43750", "AT5G49540", "AT5G59650"]

item is ["AT4G30850", "AT1G69870", "AT5G38990"]
one network is ["AT1G04310", "AT1G07860", "AT1G16170", "AT1G17280", "AT1G29060", "AT1G47640", "AT1G60940", "AT1G69870", "AT1G71140", "AT1G77350", "AT2G26180", "AT2G28250", "AT2G32380", "AT2G41705", "AT3G10980", "AT3G12180", "AT3G17000", "AT3G21580", "AT3G26370", "AT3G28220", "AT3G45290", "AT3G52070", "AT3G61260", "AT3G66654", "AT4G14455", "AT4G20790", "AT4G22890", "AT4G30500", "AT4G30850", "AT4G37680", "AT5G06320",


item is ["AT4G29930", "AT1G03130", "AT4G02770"]
one network is ["AT1G01260", "AT1G03130", "AT1G09660", "AT1G09670", "AT1G15670", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G29910", "AT1G29920", "AT1G52220", "AT1G60950", "AT2G05620", "AT2G17880", "AT2G22425", "AT2G23290", "AT2G34420", "AT3G02150", "AT3G06940", "AT3G08530", "AT3G12180", "AT3G16250", "AT3G24140", "AT3G47620", "AT3G48150", "AT3G49580", "AT3G61150", "AT4G02770", "AT4G08150", "AT4G17460", "AT4G17680", "AT4G19700", "AT4G20790", "AT4G22890", "AT4G29930", "AT4G30080", "AT4G32190", "AT4G37680", "AT4G38690", "AT5G15850", "AT5G17300", "AT5G24660", "AT5G25752", "AT5G32440", "AT5G37670", "AT5G59650", "AT5G61010", "AT5G66190"]

item is ["AT5G32440", "AT1G03130", "AT4G02770"]
one network is ["AT1G01260", "AT1G03130", "AT1G09660", "AT1G09670", "AT1G15670", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G29910", "AT1G29920", "AT1G52220", "AT1G60950", "AT2G05620", "AT2G17880", "AT2G22425",


item is ["AT3G53900", "AT3G16250", "AT4G09010"]
one network is ["AT1G03130", "AT1G14920", "AT1G22920", "AT1G29910", "AT1G29920", "AT2G01570", "AT2G17880", "AT2G23420", "AT2G30950", "AT2G33880", "AT2G34420", "AT3G02150", "AT3G16250", "AT3G47620", "AT3G48150", "AT3G53900", "AT3G62410", "AT4G02770", "AT4G09010", "AT4G17460", "AT5G15850", "AT5G17300"]

item is ["AT2G01950", "AT2G13360", "AT2G47930", "AT5G01530"]
one network is ["AT1G10090", "AT2G01950", "AT2G13360", "AT2G47930", "AT3G14415", "AT3G15710", "AT3G28220", "AT3G46090", "AT5G01530", "AT5G19120"]



[["AT2G01950", "AT2G13360", "AT3G14415", "AT3G46090", "AT5G19120"], ["AT1G10090", "AT2G01950", "AT2G13360", "AT2G47930", "AT3G14415", "AT3G15710", "AT3G46090", "AT5G01530", "AT5G19120"], ["AT1G09530", "AT1G09570", "AT1G78300", "AT2G18790", "AT2G20180", "AT2G25930", "AT2G40080", "AT2G43010", "AT3G46640", "AT4G33010", "AT4G37930", "AT5G04140"], ["AT1G01260", "AT1G03040", "AT1G03130", "AT1G03150", "AT1G04990", "AT1G05710", "AT1G08465", "AT1G08970", "AT1G09660", "AT1G09670", "AT1G10586", "AT1G14920", "AT1G15670", "AT1G20700", "AT1G20910", "AT1G22920", "AT1G26780", "AT1G29910", "AT1G29920", "AT1G30490", "AT1G36060", "AT1G42050", "AT1G48195", "AT1G48500", "AT1G52220", "AT1G53090", "AT1G60250", "AT1G60380", "AT1G62085", "AT1G63820", "AT1G69180", "AT1G70920", "AT1G71130", "AT1G72310", "AT1G72570", "AT1G74890", "AT1G74930", "AT1G75520", "AT1G76510", "AT2G01570", "AT2G05330", "AT2G14880", "AT2G17880", "AT2G18120", "AT2G22800", "AT2G23290", "AT2G23420", "AT2G25000", "AT2G27930", "AT2G30432", "AT2

In [89]:
a= object[0].get_network
a.length

item is ["AT5G19120", "AT3G46090"]
one network is ["AT2G01950", "AT2G13360", "AT3G14415", "AT3G46090", "AT5G19120"]

item is ["AT2G13360", "AT3G14415", "AT3G46090", "AT2G01950"]
one network is ["AT1G10090", "AT2G01950", "AT2G13360", "AT2G47930", "AT3G14415", "AT3G15710", "AT3G46090", "AT5G01530", "AT5G19120"]

item is ["AT4G05180", "AT4G25200"]
item is ["AT4G12800", "AT2G46820"]
item is ["AT5G04140", "AT4G37930", "AT1G78300"]
one network is ["AT1G09530", "AT1G09570", "AT1G78300", "AT2G18790", "AT2G20180", "AT2G25930", "AT2G40080", "AT2G43010", "AT3G46640", "AT4G33010", "AT4G37930", "AT5G04140"]

item is ["AT1G03130", "AT2G23290", "AT4G08150", "AT4G29930", "AT4G32190", "AT5G32440", "AT3G08530", "AT4G19700", "AT3G47620", "AT5G37670", "AT5G61010", "AT3G02150", "AT3G48150", "AT1G52220"]
one network is ["AT1G01260", "AT1G03040", "AT1G03130", "AT1G03150", "AT1G04990", "AT1G05710", "AT1G08465", "AT1G08970", "AT1G09660", "AT1G09670", "AT1G10586", "AT1G14920", "AT1G15670", "AT1G20700", "AT1G209


item is ["AT4G33010", "AT1G78300"]
one network is ["AT1G09530", "AT1G09570", "AT1G78300", "AT2G18790", "AT2G20180", "AT2G25930", "AT2G40080", "AT2G43010", "AT3G46640", "AT4G33010", "AT4G37930", "AT5G04140"]

item is ["AT1G12780", "AT4G23920"]
item is ["AT2G17880", "AT3G48150", "AT3G49580", "AT4G17680"]
one network is ["AT1G01260", "AT1G03130", "AT1G09660", "AT1G09670", "AT1G14920", "AT1G15670", "AT1G22920", "AT1G52220", "AT2G01570", "AT2G17880", "AT2G23290", "AT2G23420", "AT2G33880", "AT3G02150", "AT3G06940", "AT3G08530", "AT3G16250", "AT3G24140", "AT3G47620", "AT3G48150", "AT3G49580", "AT3G53900", "AT3G61150", "AT4G02770", "AT4G08150", "AT4G17680", "AT4G19700", "AT4G22890", "AT4G29930", "AT4G30080", "AT4G32190", "AT5G24660", "AT5G32440", "AT5G37670", "AT5G61010"]

item is ["AT3G49160", "AT3G15420", "AT2G47710", "AT2G42910", "AT2G01760", "AT4G16143", "AT1G71050", "AT2G36930"]
one network is ["AT1G71050", "AT2G01760", "AT2G36930", "AT2G42910", "AT2G47710", "AT3G15420", "AT3G49160", "AT


item is ["AT1G42970", "AT2G42580", "AT1G15780"]
one network is ["AT1G15780", "AT1G42970", "AT2G42580"]

item is ["AT5G17300", "AT1G08465", "AT1G20910", "AT3G15030", "AT5G08330", "AT1G76510", "AT1G60250", "AT1G60380", "AT1G69180", "AT1G71130", "AT2G14880", "AT2G25000", "AT2G30432", "AT2G39030", "AT2G41940", "AT2G44020", "AT2G45680", "AT2G48100", "AT3G02150", "AT3G04730", "AT3G03550", "AT3G12720", "AT3G17010", "AT5G24520", "AT4G14225", "AT4G15248", "AT3G59470", "AT3G53310", "AT3G53340", "AT2G31070", "AT1G03040", "AT3G18550", "AT3G24820", "AT3G26620", "AT3G47620", "AT3G48590", "AT4G03250", "AT4G35550", "AT5G59800", "AT5G56860", "AT5G47370", "AT5G42700", "AT5G23280", "AT5G23090", "AT5G15210"]
one network is ["AT1G01260", "AT1G03040", "AT1G03130", "AT1G03150", "AT1G04990", "AT1G05710", "AT1G08465", "AT1G08970", "AT1G09660", "AT1G09670", "AT1G10586", "AT1G14920", "AT1G15670", "AT1G20700", "AT1G20910", "AT1G22920", "AT1G26780", "AT1G29910", "AT1G29920", "AT1G30490", "AT1G36060", "AT1G42050",


item is ["AT3G54050", "AT4G09570"]
one network is ["AT1G26830", "AT1G35490", "AT1G35670", "AT3G01770", "AT3G48360", "AT3G54050", "AT4G09570", "AT4G14770"]

item is ["AT3G45780", "AT5G64330", "AT4G26090"]
one network is ["AT3G45780", "AT4G26090", "AT5G64330"]

item is ["AT2G30950", "AT1G22920", "AT2G25000"]
one network is ["AT1G03040", "AT1G08465", "AT1G14920", "AT1G20910", "AT1G22920", "AT1G32060", "AT1G60250", "AT1G60380", "AT1G69180", "AT1G71130", "AT1G76510", "AT2G01570", "AT2G14880", "AT2G23420", "AT2G25000", "AT2G30432", "AT2G30950", "AT2G31070", "AT2G33880", "AT2G39030", "AT2G41940", "AT2G44020", "AT2G45680", "AT2G48100", "AT3G02150", "AT3G03550", "AT3G04730", "AT3G12720", "AT3G15030", "AT3G16250", "AT3G17010", "AT3G18550", "AT3G24820", "AT3G26620", "AT3G26650", "AT3G47620", "AT3G48150", "AT3G48590", "AT3G53310", "AT3G53340", "AT3G53900", "AT3G59470", "AT3G62410", "AT4G03250", "AT4G14225", "AT4G15248", "AT4G35550", "AT5G08330", "AT5G15210", "AT5G17300", "AT5G23090", "AT5G23280",


item is ["AT5G43750", "AT1G17280", "AT1G14360", "AT5G35460", "AT5G47180", "AT1G29060", "AT4G20790", "AT2G22425", "AT3G10640", "AT1G70760", "AT2G43420", "AT3G12180", "AT5G04990"]
one network is ["AT1G01230", "AT1G01620", "AT1G04310", "AT1G07860", "AT1G08590", "AT1G10030", "AT1G10470", "AT1G11915", "AT1G14360", "AT1G16170", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G30080", "AT1G31420", "AT1G32210", "AT1G33100", "AT1G36050", "AT1G44960", "AT1G47640", "AT1G49230", "AT1G54320", "AT1G54350", "AT1G57540", "AT1G57550", "AT1G60940", "AT1G60950", "AT1G62200", "AT1G66340", "AT1G68380", "AT1G69870", "AT1G70760", "AT1G71140", "AT1G72300", "AT1G74370", "AT1G77110", "AT1G77350", "AT1G77380", "AT2G01420", "AT2G01970", "AT2G02020", "AT2G03620", "AT2G05620", "AT2G15970", "AT2G19580", "AT2G22425", "AT2G23093", "AT2G23095", "AT2G24170", "AT2G26180", "AT2G28250", "AT2G28960", "AT2G29020", "AT2G29180", "AT2G32380", "AT2G33250", "AT2G34250", "AT2G35520", "AT2G37360", "AT2G38120",

...


item is ["AT2G22425", "AT4G22890", "AT5G43750"]
one network is ["AT1G01260", "AT1G09660", "AT1G09670", "AT1G14360", "AT1G15670", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G60950", "AT1G69870", "AT1G70760", "AT2G05620", "AT2G22425", "AT2G23290", "AT2G43420", "AT3G06940", "AT3G08530", "AT3G10640", "AT3G12180", "AT3G24140", "AT3G28220", "AT3G47620", "AT3G48150", "AT3G49580", "AT3G61150", "AT3G61260", "AT4G02770", "AT4G17680", "AT4G19700", "AT4G20790", "AT4G22890", "AT4G29930", "AT4G30080", "AT4G37680", "AT4G38690", "AT5G04990", "AT5G24660", "AT5G25752", "AT5G32440", "AT5G35460", "AT5G37670", "AT5G38990", "AT5G43750", "AT5G47180", "AT5G59650", "AT5G61010", "AT5G66190"]

item is ["AT3G12180", "AT4G22890", "AT1G69870", "AT5G43750", "AT3G61260", "AT5G38990"]
one network is ["AT1G01260", "AT1G04310", "AT1G07860", "AT1G09660", "AT1G09670", "AT1G10470", "AT1G14360", "AT1G15670", "AT1G16170", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G47640",


item is ["AT5G06320", "AT1G69870", "AT5G38990"]
one network is ["AT1G04310", "AT1G07860", "AT1G16170", "AT1G17280", "AT1G29060", "AT1G47640", "AT1G60940", "AT1G69870", "AT1G71140", "AT1G77350", "AT2G26180", "AT2G28250", "AT2G32380", "AT2G41705", "AT3G10980", "AT3G12180", "AT3G17000", "AT3G21580", "AT3G26370", "AT3G28220", "AT3G45290", "AT3G52070", "AT3G61260", "AT3G66654", "AT4G14455", "AT4G20790", "AT4G22890", "AT4G30500", "AT4G30850", "AT4G37680", "AT5G06320", "AT5G11890", "AT5G38990", "AT5G43750", "AT5G49540", "AT5G59650"]

item is ["AT4G30850", "AT1G69870", "AT5G38990"]
one network is ["AT1G04310", "AT1G07860", "AT1G16170", "AT1G17280", "AT1G29060", "AT1G47640", "AT1G60940", "AT1G69870", "AT1G71140", "AT1G77350", "AT2G26180", "AT2G28250", "AT2G32380", "AT2G41705", "AT3G10980", "AT3G12180", "AT3G17000", "AT3G21580", "AT3G26370", "AT3G28220", "AT3G45290", "AT3G52070", "AT3G61260", "AT3G66654", "AT4G14455", "AT4G20790", "AT4G22890", "AT4G30500", "AT4G30850", "AT4G37680", "AT5G06320",


item is ["AT4G29930", "AT1G03130", "AT4G02770"]
one network is ["AT1G01260", "AT1G03130", "AT1G09660", "AT1G09670", "AT1G15670", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G29910", "AT1G29920", "AT1G52220", "AT1G60950", "AT2G05620", "AT2G17880", "AT2G22425", "AT2G23290", "AT2G34420", "AT3G02150", "AT3G06940", "AT3G08530", "AT3G12180", "AT3G16250", "AT3G24140", "AT3G47620", "AT3G48150", "AT3G49580", "AT3G61150", "AT4G02770", "AT4G08150", "AT4G17460", "AT4G17680", "AT4G19700", "AT4G20790", "AT4G22890", "AT4G29930", "AT4G30080", "AT4G32190", "AT4G37680", "AT4G38690", "AT5G15850", "AT5G17300", "AT5G24660", "AT5G25752", "AT5G32440", "AT5G37670", "AT5G59650", "AT5G61010", "AT5G66190"]

item is ["AT5G32440", "AT1G03130", "AT4G02770"]
one network is ["AT1G01260", "AT1G03130", "AT1G09660", "AT1G09670", "AT1G15670", "AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G29910", "AT1G29920", "AT1G52220", "AT1G60950", "AT2G05620", "AT2G17880", "AT2G22425",


item is ["AT3G53900", "AT3G16250", "AT4G09010"]
one network is ["AT1G03130", "AT1G14920", "AT1G22920", "AT1G29910", "AT1G29920", "AT2G01570", "AT2G17880", "AT2G23420", "AT2G30950", "AT2G33880", "AT2G34420", "AT3G02150", "AT3G16250", "AT3G47620", "AT3G48150", "AT3G53900", "AT3G62410", "AT4G02770", "AT4G09010", "AT4G17460", "AT5G15850", "AT5G17300"]

item is ["AT2G01950", "AT2G13360", "AT2G47930", "AT5G01530"]
one network is ["AT1G10090", "AT2G01950", "AT2G13360", "AT2G47930", "AT3G14415", "AT3G15710", "AT3G28220", "AT3G46090", "AT5G01530", "AT5G19120"]



67

In [ ]:
  def get_network
  ## first level interactions
    int_L1=Array.new    
    @@all_objects.each do |gene|
       if gene.interactions != [] # if there are interacions
         int_L1.push([gene.gene_id,interactions].flatten) # add that interaction as [my_gene,interaction1,interaction2, ...] 
       end
     end
    
    puts "L1 is #{int_L1}"
    
    ## second level interactions
    int_L2=Array.new
    int_L1.each do |gene|  # takes something like [my_gene, interaction1, interaction2, ...]
    
      gene[1..-1].each do |interaction| #takes each item of [interaction1, interaction2, ...]
        int_list=[]
        
        int_L1.each do |item| # per set of [my_gene, [interaction1, interaction2, ...]]
          if item[1..-1].include? interaction # if [interaction1, interaction2, ...] includes my_gene
            int_list.push([item[0]]) #add my_gene to the list
          end
        end
        
        int_list.flatten!.uniq! # un-nest arrays and remove duplicates
        if int_list.length > 1 #there is interaction with 2 genes and therefore is part of a network
          int_L2.push([interaction,int_list].flatten)
        end
        
      end
    end
    
    puts ""
    puts "L2 is #{int_L2}"
    puts ""
    
    ## merge and make that array a local variable so it can be called later on
    all=[int_L1,int_L2].flatten!(1)
    puts "all is #{all}"
    puts ""
    
     
    ## create network's arrays
    
    networks=Array.new
    all.each do |item| # get one array of interactions, for example [gene1 , gene2, gene3]
      net=Array.new
      
      item.each do |elem| # get one of those genes, for example gene1
        (0..all.length-1).each do |i| #iterate over the arrays again (like all[0] = [gene1 , gene2, gene3])
          if all[i].any? elem #if the array contains that gene (for example gene1 is in all[0])
            net.push(all[i].flatten) # include that array of interactions to my new array 
          end
        end
      end
      net.flatten!.uniq!.sort!
      if net.length > 2 
        networks.push(net)
      end
    end
    
    networks.uniq! #remove duplicated networks
    return networks
      
  end

In [16]:
  address="http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/query/AT2G23290?format=tab25"
  response = fetch(address)
  interactions_array=Array.new
  puts "get_interactions function has been called"
  
a=[]

  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = body.split("\n").to_a #first I split the rows into different arrays
    (0..data.length-1).each do |i|
      
      data[i] = data[i].split("\t") #then I create an array's element per each tab separated value 
      #data[i] = data[i][4..5] #these columns of the tab25 format cointain the gene locus name
      
      intact_miscore = data[i][14].split(":")[1]
      a.push(intact_miscore)
      
    end    
  end

get_interactions function has been called


0..68

In [21]:
a=["taxid:3702(arath)|taxid:3702('Arabidopsis thaliana (Mouse-ear cress)')"],["taxid:9913(bovin)|taxid:9913('Bos taurus (Bovine)')"]


[["taxid:3702(arath)|taxid:3702('Arabidopsis thaliana (Mouse-ear cress)')"], ["taxid:9913(bovin)|taxid:9913('Bos taurus (Bovine)')"]]

In [31]:
a[0][0]

"taxid:3702(arath)|taxid:3702('Arabidopsis thaliana (Mouse-ear cress)')"

In [39]:
if a[0][0].include? "3702"
  puts "YASS"
end

YASS


In [41]:
a=[1,2,3,4,5,6,7,8,9]

a.each do |w|
  if w == 2
    next
  end
  puts w
end

1
3
4
5
6
7
8
9


[1, 2, 3, 4, 5, 6, 7, 8, 9]